In [1]:
#segmentation
import nltk
nltk.download('punkt')
#Tokenize text to sentences
from nltk.tokenize import sent_tokenize
#Tokenize sentence in the text to words
from nltk.tokenize import word_tokenize
#Remove the stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
#Perfomr Steming and lemmatization
from nltk.stem.porter import PorterStemmer
nltk.download('wordnet') # download for lemmatization
nltk.download('omw-1.4')
from nltk.stem.wordnet import WordNetLemmatizer
import pandas as pd   

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\boseb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\boseb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\boseb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\boseb\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [39]:
import re
def standardize_text(text):
    text = text.lower()  # Lowercase
    # Segmentation
    sentences = sent_tokenize(text)
    words = []
    lemmatized_all = []
    # Remove punctuation
    for sentence in sentences:
        new_text = re.sub(r"[^a-zA-Z0-9]", " ", sentence) 
        new_text = re.sub(r"x+\s","", new_text)
        # Tokenization
        words = word_tokenize(new_text)
        # StopWords
        words = [w for w in words if w not in stopwords.words("english")]
        # Lemmatization
        lemmatized = [WordNetLemmatizer().lemmatize(w) for w in words]
        # print('###################################')
        # print(f"Lemmatized String {lemmatized}")
        lemmatized_all.extend(lemmatized)       
    # print('###################################')
    # print(f"Lemmatized All {lemmatized_all}")
    unique_words = list(set(lemmatized_all))
    # print('###################################')
    # print(f"Unique Words : {unique_words}")
    return ' '.join(unique_words)

In [40]:
text = """My XXXX  instructed me to write a letter because I am a XXXX  of XXXX XXXX.
I am in a desperate need to fix this problem since I am trying to get things in order The Accounts that I put below are due to the XXXX XXXX  violation that has nearly ruined my life Please remove these accounts so I can start taking steps to getting my life back together. 
Thank you so much for assisting me and offering these resources to XXXX  like me. 
ACCOUNT THAT ARE FRAUD DUE TO XXXX XXXX  : XXXX. XXXX XXXXXXXX XXXX XXXX  opened XX/XX/XXXX XXXX. XXXX XXXXXXXX XXXX XXXX opened XX/XX/XXXX XXXX. XXXX  XXXX XXXX opened XX/XX/XXXX XXXX. The XXXX Inquiries Below | The only inquiry I did was the XXXX  XXXX XXXX in XXXX, XXXX"""

standardize_text(text)

'remove put start much inquiry resource get fraud letter thing ruined like life please due opened instructed assisting offering violation account write nearly problem xxxx need getting desperate back step taking fithis together thank order since trying'

In [41]:
file_loc = "../Resources/ModelData/nlp_customer_complaint_data.csv"
customer_complaints_df = pd.read_csv(file_loc)
customer_complaints_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316237 entries, 0 to 316236
Data columns (total 2 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   Consumer complaint narrative  316237 non-null  object
 1   Product                       316237 non-null  object
dtypes: object(2)
memory usage: 4.8+ MB


In [44]:
customer_complaints_df.head(3)
customer_complaints_df['Consumer complaint narrative'] = customer_complaints_df['Consumer complaint narrative'].apply(standardize_text)
customer_complaints_df.head(3)

KeyboardInterrupt: 